In [21]:
import requests
from bs4 import BeautifulSoup

base_url = "https://www.cardrush-pokemon.jp/phone/new"
url = f"{base_url}/new?page=1"

def extract_goods_and_prices(soup):
    results = []

    item_data_divs = soup.find_all('div', class_='item_data')

    for div in item_data_divs:
        goods_name = div.find('span', class_='goods_name')
        selling_price = div.find('p', class_='selling_price')

        if goods_name and selling_price:
            result = (goods_name.text.strip(), selling_price.text.strip())
            results.append(result)

    return results

def fetch_and_parse(url):
    response = requests.get(url)

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        return soup
    else:
        print(f"Failed to fetch the page, status code: {response.status_code}")
        return None

# Fetch the first page
soup = fetch_and_parse(url)

if soup:
    # Extract goods_name and selling_price from the first page
    all_results = extract_goods_and_prices(soup)

    # Find the last page number
    pagebottom_div = soup.find('div', id='pagerbottom')

    if pagebottom_div:
        last_page_link = pagebottom_div.find_all('a', class_='pager_btn')[-2]
        last_page_number = int(last_page_link['href'].split('=')[-1])
        print(last_page_link)
        print(last_page_number)
        
        
        # Loop through the remaining pages
        for page_number in range(2, last_page_number + 1):
            print(page_number)
            page_url = f"{base_url}/new?page={page_number}"
            soup = fetch_and_parse(page_url)

            if soup:
                # Extract goods_name and selling_price from the current page
                results = extract_goods_and_prices(soup)
                all_results.extend(results)

    # Print the extracted data
    for i, result in enumerate(all_results):
        print(f"Result {i + 1}:")
        print(f"Goods Name: {result[0]}")
        print(f"Selling Price: {result[1]}")
        print("\n")




<a class="to_last_page pager_btn" href="/phone/new/new?page=34">34</a>
34
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
Result 1:
Goods Name: 《No.405》1パック10000円オリパ！！【No.405】(※ラストワン有){￥10000}
Selling Price: 10,000円(税込)


Result 2:
Goods Name: 《No.404》1パック1500円最新弾オリパ！！！【No.404】(※ラストワン有){￥1500}
Selling Price: 1,500円(税込)


Result 3:
Goods Name: 《No.403》PSAオリパ！！1口10万円150口限定【No.403】(※ラストワン有){￥100000}
Selling Price: 100,000円(税込)


Result 4:
Goods Name: ☆SALE☆勇気のおまもり【U】{067/071}
Selling Price: 180円(税込)


Result 5:
Goods Name: ☆SALE☆すごいつりざお【U】{066/071}
Selling Price: 220円(税込)


Result 6:
Goods Name: ☆SALE☆リバーサルエネルギー【U】{071/071}
Selling Price: 120円(税込)


Result 7:
Goods Name: ☆SALE☆ナンジャモ【U】{069/071}
Selling Price: 580円(税込)


Result 8:
Goods Name: ディンルーex【UR】{097/071}
Selling Price: 1,980円(税込)


Result 9:
Goods Name: パオジアンex【UR】{097/071}
Selling Price: 3,780円(税込)


Result 10:
Goods Name: スーパーエネルギー回収【UR】{098/071}
Selling Price: 3,280円(税込)


Result 11:
G

Result 701:
Goods Name: デデンネ【C】{045/098}
Selling Price: 50円(税込)


Result 702:
Goods Name: ヤジロン【C】{047/098}
Selling Price: 30円(税込)


Result 703:
Goods Name: ネンドール【U】{048/098}
Selling Price: 30円(税込)


Result 704:
Goods Name: アノプス【C】{049/098}
Selling Price: 30円(税込)


Result 705:
Goods Name: アーマルド【U】{050/098}
Selling Price: 30円(税込)


Result 706:
Goods Name: ヤンチャム【C】{052/098}
Selling Price: 30円(税込)


Result 707:
Goods Name: イワンコ【C】{053/098}
Selling Price: 30円(税込)


Result 708:
Goods Name: ルガルガン【U】{054/098}
Selling Price: 30円(税込)


Result 709:
Goods Name: イシヘンジン【U】{055/098}
Selling Price: 30円(税込)


Result 710:
Goods Name: チョロネコ【C】{057/098}
Selling Price: 30円(税込)


Result 711:
Goods Name: レパルダス【U】{058/098}
Selling Price: 30円(税込)


Result 712:
Goods Name: ゴロンダ【U】{059/098}
Selling Price: 30円(税込)


Result 713:
Goods Name: クズモー【C】{060/098}
Selling Price: 30円(税込)


Result 714:
Goods Name: ドラミドロ【U】{061/098}
Selling Price: 30円(税込)


Result 715:
Goods Name: モルペコ【C】{062/098}
Selling Price: 50円(税込)


R

Result 1326:
Goods Name: コロトックV【CSR】{213/184}
Selling Price: 780円(税込)


Result 1327:
Goods Name: ヤレユータン【CHR】{212/184}
Selling Price: 120円(税込)


Result 1328:
Goods Name: ポワルン【CHR】{211/184}
Selling Price: 180円(税込)


Result 1329:
Goods Name: イーブイ【CHR】{210/184}
Selling Price: 580円(税込)


Result 1330:
Goods Name: クリムガン【CHR】{209/184}
Selling Price: 280円(税込)


Result 1331:
Goods Name: ドータクン【CHR】{208/184}
Selling Price: 280円(税込)


Result 1332:
Goods Name: ガラルタチフサグマ【CHR】{207/184}
Selling Price: 120円(税込)


Result 1333:
Goods Name: ヘルガー【CHR】{206/184}
Selling Price: 180円(税込)


Result 1334:
Goods Name: アリアドス【CHR】{205/184}
Selling Price: 120円(税込)


Result 1335:
Goods Name: タイレーツ【CHR】{204/184}
Selling Price: 380円(税込)


Result 1336:
Goods Name: ナゲツケサル【CHR】{203/184}
Selling Price: 180円(税込)


Result 1337:
Goods Name: イワンコ【CHR】{202/184}
Selling Price: 180円(税込)


Result 1338:
Goods Name: マホイップ【CHR】{201/184}
Selling Price: 180円(税込)


Result 1339:
Goods Name: デデンネ【CHR】{200/184}
Selling Price: 480円(税込)


Resu


Result 1951:
Goods Name: 〔状態B〕ツツジ(ミラー)【-】{157/172}
Selling Price: 50円(税込)


Result 1952:
Goods Name: 〔状態B〕チェレンの気くばり(ミラー)【-】{156/172}
Selling Price: 50円(税込)


Result 1953:
Goods Name: 〔状態B〕シンオウの仲間たち(ミラー)【-】{154/172}
Selling Price: 50円(税込)


Result 1954:
Goods Name: 〔状態B〕シロナの覇気(ミラー)【-】{153/172}
Selling Price: 50円(税込)


Result 1955:
Goods Name: 〔状態B〕ザクロ(ミラー)【-】{152/172}
Selling Price: 50円(税込)


Result 1956:
Goods Name: 〔状態B〕キバナ(ミラー)【-】{151/172}
Selling Price: 50円(税込)


Result 1957:
Goods Name: 〔状態B〕カミツレのきらめき(ミラー)【-】{150/172}
Selling Price: 50円(税込)


Result 1958:
Goods Name: 〔状態B〕アクロマの実験(ミラー)【-】{148/172}
Selling Price: 80円(税込)


Result 1959:
Goods Name: 〔状態B〕ロストスイーパー(ミラー)【-】{144/172}
Selling Price: 50円(税込)


Result 1960:
Goods Name: 〔状態B〕レベルボール(ミラー)【-】{143/172}
Selling Price: 50円(税込)


Result 1961:
Goods Name: 〔状態B〕レスキューキャリー(ミラー)【-】{142/172}
Selling Price: 50円(税込)


Result 1962:
Goods Name: 〔状態B〕パワータブレット(ミラー)【-】{139/172}
Selling Price: 50円(税込)


Result 1963:
Goods Name: 〔状態B〕ダメージポンプ(ミラー)【